In [1]:
# System imports
import os
import sys
import time
import datetime
from tqdm import tqdm
from typing_extensions import Self, Any      # For Python 3.10
# from typing import Self, Any               # For Python >3.11

from pathlib import Path

# Data manipulation imports
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Data visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning imports
import tensorflow as tf
from keras.ops import add
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, Lambda, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras import regularizers                                                                           # For L2 regularization
#import visualkeras

# Evaluation imports
from keras.metrics import CategoricalAccuracy, AUC, F1Score, Precision, Recall

# Other imports
from itertools import product

# Set the style of the visualization
pd.set_option('future.no_silent_downcasting', True)   # use int instead of float in DataFrame
pd.set_option("display.max_columns", None)            # display all columns

# Disable warnings (FutureWarning)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set random seed for reproducibility
np.random.seed(2025)

2025-04-14 17:04:50.644248: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-14 17:04:50.652641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744646690.663492 1017959 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744646690.666873 1017959 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 17:04:50.677937: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("GPU Device Name:", tf.test.gpu_device_name())                                # (if error in Google Colab: Make sure your Hardware accelerator is set to GPU. 
                                                                                    # Runtime > Change runtime type > Hardware Accelerator)

TensorFlow Version: 2.18.0
Is TensorFlow built with CUDA? True
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Device Name: /device:GPU:0


I0000 00:00:1744646693.696920 1017959 gpu_device.cc:2022] Created device /device:GPU:0 with 8782 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
# Get build information from TensorFlow
build_info = tf.sysconfig.get_build_info()

print("TensorFlow version:", tf.__version__)
print("Python version:", sys.version)
print("CUDA version:", build_info.get("cuda_version", "Not available"))
print("cuDNN version:", build_info.get("cudnn_version", "Not available"))

TensorFlow version: 2.18.0
Python version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
CUDA version: 12.5.1
cuDNN version: 9


In [10]:
# Import custom module for importing data, visualization, and utilities
import utilities

In [11]:
# Define the path to the data
train_dir = Path("../data/RareSpecies_Split/train")
val_dir = Path("../data/RareSpecies_Split/val")
test_dir = Path("../data/RareSpecies_Split/test")

# For Google Collab
# train_dir = Path("/content/RareSpecies_Split/train")
# val_dir = Path("/content/RareSpecies_Split/val")
# test_dir = Path("/content/RareSpecies_Split/test")

In [12]:
# Image Generators 
n_classes = 202                                     # Number of classes (we already know this based on previous notebook)
image_size = (224, 224)                             # Image size (224x224)
img_height, img_width = image_size                  # Image dimensions
batch_size = 64                                     # Batch size
input_shape = (img_height, img_width, 3)            # Input shape of the model
value_range = (0.0, 1.0)                            # Range of pixel values

In [13]:
# Get class names from directory
class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}

# Import the image dataset from the directory
from utilities import load_images_from_directory
train_datagen, val_datagen, test_datagen = load_images_from_directory(train_dir, val_dir, test_dir,
                                                                      labels='inferred', label_mode='categorical',
                                                                      class_names=class_names, color_mode='rgb',
                                                                      batch_size=batch_size, image_size=image_size, seed=2025, 
                                                                      interpolation='bilinear', crop_to_aspect_ratio=False, pad_to_aspect_ratio=False)

print(f"\nLoaded: Train ({train_datagen.cardinality().numpy() * batch_size}), "
        f"Val ({val_datagen.cardinality().numpy() * batch_size}), "
        f"Test ({test_datagen.cardinality().numpy() * batch_size})")

Found 9586 files belonging to 202 classes.


I0000 00:00:1744646955.215843 1017959 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8782 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 1198 files belonging to 202 classes.
Found 1199 files belonging to 202 classes.

Loaded: Train (9600), Val (1216), Test (1216)


In [14]:
# Check the shape of the data (batch_size, img_width, img_height, 3)
for x, y in train_datagen.take(1):
    print("Train batch shape:", x.shape, y.shape)
for x, y in val_datagen.take(1):
    print("Val batch shape:", x.shape, y.shape)
for x, y in test_datagen.take(1):
    print("Test batch shape:", x.shape, y.shape)

Train batch shape: (64, 224, 224, 3) (64, 202)
Val batch shape: (64, 224, 224, 3) (64, 202)


2025-04-14 17:09:20.073815: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-14 17:09:20.206682: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Test batch shape: (64, 224, 224, 3) (64, 202)


In [16]:
from tensorflow.keras.applications.convnext import ConvNeXtTiny

class RareSpeciesCNN(Model):
    """Custom CNN based on ConvNeXt architecture for rare species classification without pretrained weights."""

    def __init__(self, n_classes=202,
                 apply_grayscale=False,
                 apply_contrast=True, contrast_factor=1.3,
                 apply_saturation=True, saturation_factor=1.2):
        super().__init__()

        # Preprocessing configuration
        self.apply_grayscale = apply_grayscale
        self.apply_contrast = apply_contrast
        self.apply_saturation = apply_saturation

        # Preprocessing layers
        self.rescale_layer = Rescaling(scale=1/255.0, name="Rescale_Layer")

        if self.apply_contrast:
            self.contrast_layer = Lambda(
                lambda x: tf.image.adjust_contrast(x, contrast_factor),
                name='Adjust_Contrast'
            )

        if self.apply_saturation:
            self.saturation_layer = Lambda(
                lambda x: tf.image.adjust_saturation(x, saturation_factor),
                name='Adjust_Saturation'
            )

        if self.apply_grayscale:
            self.grayscale_layer = Lambda(
                lambda x: tf.image.rgb_to_grayscale(x),
                name='RGB_to_Grayscale'
            )
            self.grayscale_to_rgb_layer = Lambda(
                lambda x: tf.image.grayscale_to_rgb(x),
                name='Grayscale_to_RGB'
            )

        # ConvNeXtTiny Backbone (WITHOUT pretrained weights)
        self.convnext_base = ConvNeXtTiny(
            include_top=False,
            weights=None,   # No pretrained weights
            input_shape=(224, 224, 3)
        )

        # Classification Head
        self.global_avg_pool = GlobalAveragePooling2D(name="Global_Average_Pooling")
        self.dropout = Dropout(0.6, name="Dropout_Layer")
        self.dense_hidden = Dense(256, activation='relu', name="Dense_Hidden")
        self.dense_output = Dense(n_classes, activation='softmax', name="Output_Layer")

    def call(self, inputs, training=False):
        # Apply preprocessing
        x = self.rescale_layer(inputs)

        if self.apply_contrast:
            x = self.contrast_layer(x)

        if self.apply_saturation:
            x = self.saturation_layer(x)

        if self.apply_grayscale:
            x = self.grayscale_layer(x)
            x = self.grayscale_to_rgb_layer(x)

        # ConvNeXt backbone
        x = self.convnext_base(x, training=training)

        # Classification head
        x = self.global_avg_pool(x)
        x = self.dropout(x, training=training)
        x = self.dense_hidden(x)
        outputs = self.dense_output(x)

        return outputs

# Instantiate model without pretrained weights
model = RareSpeciesCNN(
    n_classes=202,
    apply_grayscale=False,
    apply_contrast=True, contrast_factor=1.3,
    apply_saturation=True, saturation_factor=1.2,
)

# Build and summarize the model
inputs = Input(shape=(224, 224, 3))
_ = model.call(inputs)
model.summary()


Model: "rare_species_cnn_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Rescale_Layer (Rescaling)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Adjust_Contrast (Lambda)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Adjust_Saturation (Lambda)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convnext_tiny (Functional)      │ (None, 7, 7, 768)      │    27,820,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Global_Average_Pooling          │ (None, 768)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_Layer (Dropout)         │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_Hidden (Dense)            │ (None, 256)            │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 202)            │        51,914 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,068,906 (107.07 MB)

 Trainable params: 28,068,906 (107.07 MB)

 Non-trainable params: 0 (0.00 B)